In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 936 kB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=60a1a5af66fdb99ae4b2be0593dfec956d940a7af6564d5cd3b859f2a0136d36
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [2]:
import os

import numpy as np
import pandas as pd
import cloudpickle

from transformers import XLMRobertaModel, XLMRobertaTokenizer
import torch

from sentence_transformers import SentenceTransformer, util

2021-11-03 14:57:45.498141: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-03 14:57:45.498262: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
df = pd.read_csv('../input/combiner-target-dataset-0-of-5/final_matchings.csv', keep_default_na=False)

df

,image_url,page_title,caption_reference_description,count,spaced_filename,spaced_filename_translit,page_title_translit,caption_translit,page_title_lang,page_title_lang_p,...,page_title_contains_digit,undigit_page_title,target,target_id,caption_translation,final_caption,PREfinal_caption,page_title_translation,final_page_title,PREfinal_page_title
0,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens's Justitia, 1663-1665, Museumsber...",4,Jurgen ovens justitia,,Juergen Ovens,"Juergen Ovens's Justitia, 1663-1665, Museumsbe...",en,0.124504,...,False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens's Justitia, 16...",0,,"Jürgen Ovens's Justitia, 1663-1665, Museumsber...","Jürgen Ovens's Justitia, 1663-1665, Museumsber...",,Jürgen Ovens,Jürgen Ovens
1,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Jürgen Ovens: Justitia, \n 1663-1665, Museumsb...",4,Jurgen ovens justitia,,Juergen Ovens,"Juergen Ovens: Justitia, \n 1663-1665, Museums...",en,0.124504,...,False,Jürgen Ovens,"Jürgen Ovens [SEP] Jürgen Ovens: Justitia, \n ...",1,,"Jürgen Ovens: Justitia, \n 1663-1665, Museumsb...","Jürgen Ovens: Justitia, \n 1663-1665, Museumsb...",,Jürgen Ovens,Jürgen Ovens
2,https://upload.wikimedia.org/wikipedia/commons...,"Овенс, Юрген","Юрген Овенс Юстиция, 1663—1665, Музей Фленсбур...",4,Jurgen ovens justitia,,"Ovens, Jurgen","Jurgen Ovens Justitsija, 1663—1665, Muzej Flen...",ru,0.647271,...,False,"Овенс, Юрген","Овенс, Юрген [SEP] Юрген Овенс Юстиция, 1663—1...",2,"Jurgen Owens Justice, -, Flensburg Museum. Owe...","Jurgen Owens Justice, -, Flensburg Museum. Owe...","Jurgen Owens Justice, -, Flensburg Museum. Owe...","Owens, Jürgen","Owens, Jürgen","Owens, Jürgen"
3,https://upload.wikimedia.org/wikipedia/commons...,Jürgen Ovens,"Justitia aus Schloss Gottorf, um 1665, Museums...",4,Jurgen ovens justitia,,Juergen Ovens,,en,0.124504,...,False,Jürgen Ovens,Jürgen Ovens [SEP] Justitia aus Schloss Gottor...,3,"Justitia from Gottorf Castle, um, Museumsberg ...","Justitia from Gottorf Castle, um, Museumsberg ...","Justitia from Gottorf Castle, um, Museumsberg ...",,Jürgen Ovens,Jürgen Ovens
4,https://upload.wikimedia.org/wikipedia/commons...,Andréi Amalrik,"Andréi Amalrik, 1976",4,Andrei Amalrik 1976,,Andrei Amalrik,"Andrei Amalrik, 1976",es,0.742067,...,False,Andréi Amalrik,"Andréi Amalrik [SEP] Andréi Amalrik, 1976",4,"Andrei Amalrik,","Andrei Amalrik,","Andrei Amalrik,",Andrei Amalrik,Andrei Amalrik,Andrei Amalrik
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301659,https://upload.wikimedia.org/wikipedia/commons...,National Gallery Prague,Veletržní palác (the Trade Fair Palace) houses...,4,Praha Veletržní palác jih,Praha Veletržni palac jih,,Veletržni palac (the Trade Fair Palace) houses...,en,0.686658,...,False,National Gallery Prague,National Gallery Prague [SEP] Veletržní palác ...,301659,,Veletržní palác (the Trade Fair Palace) houses...,Veletržní palác (the Trade Fair Palace) houses...,,National Gallery Prague,National Gallery Prague
301660,https://upload.wikimedia.org/wikipedia/commons...,San Diego Convention Center,View of the center from the San Diego Bay,4,San Diego Convention Center,,,,en,0.576476,...,False,San Diego Convention Center,San Diego Convention Center [SEP] View of the ...,301660,,View of the center from the San Diego Bay,View of the center from the San Diego Bay,,San Diego Convention Center,San Diego Convention Center
301661,https://upload.wikimedia.org/wikipedia/commons...,San Diego Convention Center,View of the center from the San Diego Bay,4,San Diego Convention Center,,,,en,0.576476,...,False,San Diego Convention Center,San Diego Convention Center [SEP] View of the ...,301661,,View of the center from the San Diego Bay,View of the center from the San Diego Bay,,San Diego Convention Center,San Diego Convention Center
301662,https://upload.wikimedia.org/wikipedia/commons...,1996 Republican National Convention,The San Diego Convention Center was the 

### Loading Model

In [4]:
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Retrieving Embeddings

In [5]:
page_title_embeddings = model.encode(df['page_title'].values)

print(page_title_embeddings.shape)

with open('page_title_embeddings.data', 'wb') as file:
    cloudpickle.dump(page_title_embeddings, file)

Batches:   0%|          | 0/9427 [00:00<?, ?it/s]

(301664, 768)


In [6]:
prefinal_page_title_embeddings = model.encode(df['PREfinal_page_title'].values)

print(prefinal_page_title_embeddings.shape)

with open('prefinal_page_title_embeddings.data', 'wb') as file:
    cloudpickle.dump(prefinal_page_title_embeddings, file)

Batches:   0%|          | 0/9427 [00:00<?, ?it/s]

(301664, 768)
